In [14]:
import os
import random
import re
import string

import nltk
import numpy as np
import pandas as pd

from gensim.models import Word2Vec

from nltk import word_tokenize
from nltk.corpus import stopwords


from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

nltk.download("stopwords")
nltk.download("punkt")

SEED = 42
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/daniel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/daniel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
def clean_text(text, tokenizer, stopwords):
    """Pre-process text and generate tokens
    """
    text = str(text).lower()  # Lowercase words
    text = re.sub(r"\[(.*?)\]", "", text)  # Remove [+XYZ chars] in content
    text = re.sub(r"\s+", " ", text)  # Remove multiple spaces in content
    text = re.sub(r"\w+…|…", "", text)  # Remove ellipsis (and last word)
    text = re.sub(r"(?<=\w)-(?=\w)", " ", text)  # Replace dash between words
    text = re.sub(
        f"[{re.escape(string.punctuation)}]", "", text
    )  # Remove punctuation

    tokens = tokenizer(text)  # Get tokens from text
    tokens = [t for t in tokens if not t in stopwords]  # Remove stopwords
    tokens = ["" if t.isdigit() else t for t in tokens]  # Remove digits
    tokens = [t for t in tokens if len(t) > 1]  # Remove short tokens
    return tokens

In [7]:
df_raw = pd.read_json("/Users/daniel/Desktop/STATAPP/Statap-panels/temp_dataset.json")
df = df_raw.copy()
for col in text_columns:
    df[col] = df[col].astype(str).fillna("")

# Combine all text fields into a single text column
df["text"] = df[text_columns].apply(lambda x: " | ".join(x), axis=1)
df.head()

,history,question,query_modifier_question,generated_answer,answer,text
0,"[{'role': 'user', 'content': 'What are the mai...",What other sectors are they involved in?,What other sectors is Bouygues involved in bes...,"Besides construction, real estate, media, and ...","Besides construction and real estate, Bouygues...","[{'role': 'user', 'content': 'What are the mai..."
1,"[{'role': 'user', 'content': 'What is employee...",How did it help against Vincent Bolloré's atte...,How did employee shareholding at Bouygues help...,"In 1998, Bouygues' employee shareholding helpe...",The employee shareholding helped counter Vince...,"[{'role': 'user', 'content': 'What is employee..."
2,"[{'role': 'user', 'content': 'Where is the hea...",And what about Bouygues Construction?,Where is the headquarters of Bouygues Construc...,"The headquarters of Bouygues Construction, kno...","The headquarters of Bouygues Construction, kno...","[{'role': 'user', 'content': 'Where is the hea..."
3,"[{'role': 'user', 'content': 'What happens whe...",How much money does this generate in France?,How much money do search engines earn from spo...,Search engines in France earn approximately 2....,"In France, this generates about 2.4 billion eu...","[{'role': 'user', 'content': 'What happens whe..."
4,"[{'role': 'user', 'content': 'How can I instal...",Is there any cost involved?,Is there any cost involved in installing Lilo ...,Not enough information is available about the ...,"No, using Lilo in a business setting is simple...","[{'role': 'user', 'content': 'How can I instal..."


In [15]:
# Define stopwords (use sklearn's built-in stopwords)
custom_stopwords = set(ENGLISH_STOP_WORDS) | {"history", "question", "query", "answer"}

# Tokenization and cleaning function (without NLTK)
def clean_and_tokenize(text):
    tokens = text.lower().split()  # Lowercase & split on whitespace
    tokens = [word for word in tokens if word.isalnum() and word not in custom_stopwords]  # Remove punctuation & stopwords
    return tokens

# Apply tokenization without using nltk
df["tokens"] = df["text"].astype(str).apply(clean_and_tokenize)

# Print dataset sizes
print(f"Original dataframe: {df_raw.shape}")
print(f"Pre-processed dataframe: {df.shape}")

Original dataframe: (144, 5)
Pre-processed dataframe: (144, 7)


In [ ]:
w2v_model = Word2Vec(sentences=df["tokens"], vector_size=100, window=5, min_count=2, workers=4)

# Convert documents to feature vectors by averaging word embeddings
def document_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

# Create document embeddings
df["doc_vector"] = df["tokens"].map(lambda x: document_vector(x, w2v_model))

# Stack document vectors into a matrix
X = np.vstack(df["doc_vector"].values)

# Apply K-Means clustering
num_clusters = 3 # Adjust based on your dataset
kmeans = MiniBatchKMeans(n_clusters=num_clusters, random_state=42, n_init=10)
df["cluster"] = kmeans.fit_predict(X)

# Print results
print(df[["text", "cluster"]])

# Save clustered dataset
df.to_csv("clustered_data.csv", index=False)


                                                  text  cluster
0    [{'role': 'user', 'content': 'What are the mai...        9
1    [{'role': 'user', 'content': 'What is employee...        6
2    [{'role': 'user', 'content': 'Where is the hea...        0
3    [{'role': 'user', 'content': 'What happens whe...        9
4    [{'role': 'user', 'content': 'How can I instal...        9
..                                                 ...      ...
139  [{'role': 'user', 'content': 'What is the main...        9
140  [{'role': 'user', 'content': 'What actions hav...        0
141  [{'role': 'user', 'content': 'What does Lyon A...        0
142  [{'role': 'user', 'content': "What is Lyon Aér...        0
143  [{'role': 'user', 'content': "What is Lyon air...        9

[144 rows x 2 columns]


In [24]:
# Sort the dataframe by cluster labels
df_sorted = df.sort_values(by="cluster")

# Display a few sample rows from each cluster
for cluster in range(num_clusters):
    print(f"Cluster {cluster}:")
    display(df_sorted[df_sorted["cluster"] == cluster].head(5))
    print("\n")

Cluster 0:


,history,question,query_modifier_question,generated_answer,answer,text,tokens,doc_vector,cluster
98,"[{'role': 'user', 'content': 'What should I do...",And if my baggage is damaged?,What should I do if my baggage is damaged at L...,"If your baggage is damaged at Lyon Airport, re...","If your baggage is damaged, you should seek in...","[{'role': 'user', 'content': 'What should I do...","[lose, item, lyon, lose, item, lyon, contact, ...","[3.424577e-05, 0.0043963077, 0.0009705579, -0....",0
100,"[{'role': 'user', 'content': 'Is this service ...",Where is it located?,Where is the service located?,I am unable to provide the location of the ser...,The service is located outside.,"[{'role': 'user', 'content': 'Is this service ...","[service, service, service, unable, provide, l...","[0.0012432816, 0.0045177033, 0.0021609832, -2....",0
91,"[{'role': 'user', 'content': 'How can I get fr...",What about traveling to the Alps?,What are the travel options from Lyon Airport ...,"From Lyon Airport, you can travel to the Alps ...",You can travel to the Alps from Lyon Airport w...,"[{'role': 'user', 'content': 'How can I get fr...","[lyon, airport, city, connects, lyon, center, ...","[0.0008213088, 0.0018248743, 0.0004122288, 0.0...",0
95,"[{'role': 'user', 'content': 'Who manages the ...",Where is their office located?,Where is Alyzia's office for Easyjet baggage s...,Alyzia's office for Easyjet baggage service is...,Alyzia's office is located in the baggage clai...,"[{'role': 'user', 'content': 'Who manages the ...","[manages, baggage, service, baggage, service, ...","[0.0016431212, 0.0035506093, -0.00038029862, -...",0
93,"[{'role': 'user', 'content': 'Where is the Air...",What about the other airlines?,What is the location of baggage services for o...,The baggage service for other airlines is mana...,"For Corsair, Easyjet, Royal Air Maroc, Royal J...","[{'role': 'user', 'content': 'Where is the Air...","[air, france, baggage, service, air, france, b...","[0.0016661871, 0.0027206542, 0.0011027243, -0....",0




Cluster 1:


,history,question,query_modifier_question,generated_answer,answer,text,tokens,doc_vector,cluster
82,"[{'role': 'user', 'content': ""What was the rol...",How many ministerial departments were there du...,How many ministerial departments existed durin...,"During the French Revolution, there were gener...","During the Revolution, there were generally si...","[{'role': 'user', 'content': ""What was the rol...","[role, ministers, executors, similar, predeces...","[0.0004062135, 0.0012135122, -0.00035362944, 0...",1
83,"[{'role': 'user', 'content': 'What was the Fre...",Where was the general headquarters established?,Where was the French army's general headquarte...,The French army's general headquarters was est...,Napoleon established his general headquarters ...,"[{'role': 'user', 'content': 'What was the Fre...","[french, army, preparing, french, army, prepar...","[0.00023241593, 0.0024841188, -0.0003751751, -...",1
74,"[{'role': 'user', 'content': 'What did the emp...",What did he order next?,What did the emperor order after spreading the...,Not enough information is available to answer ...,He ordered nine battalions of the imperial gua...,"[{'role': 'user', 'content': 'What did the emp...","[did, emperor, motivate, emperor, spread, rumo...","[0.0016922804, 0.00040191793, 0.0015649986, 0....",1
22,"[{'role': 'user', 'content': 'When was COVID-1...",How did the EU manage to overcome it?,How did the European Union manage to overcome ...,The European Union overcame COVID-19 through c...,The EU successfully overcame the COVID-19 pand...,"[{'role': 'user', 'content': 'When was COVID-1...","[declared, longer, global, health, 5, declared...","[-0.00072067423, 0.00068698835, 0.00018260661,...",1
72,"[{'role': 'user', 'content': 'Who is mentioned...",What did he lose?,What did Napoléon lose?,"Napoléon lost his empire, his dynasty, and his...",Napoléon lost the three scrutins organized dur...,"[{'role': 'user', 'content': 'Who is mentioned...","[mentioned, mentioned, did, did, napoléon, nap...","[-0.0012891497, -0.0036646966, 0.00062188256, ...",1




Cluster 2:


...

,history,question,query_modifier_question,generated_answer,answer,text,tokens,doc_vector,cluster
133,"[{'role': 'user', 'content': 'How many passeng...",And how many movements were there?,How many aircraft movements were there at Atla...,Not enough information is available to determi...,"There were 8,109 movements at the Atlantic Cit...","[{'role': 'user', 'content': 'How many passeng...","[passengers, did, atlantic, city, internationa...","[0.0010982389, -0.00047543034, 0.0013788722, -...",7
116,"[{'role': 'user', 'content': 'What is the ener...",What was the initial goal?,What was the initial energy savings goal of th...,The initial energy savings goal of the City of...,The initial goal was to achieve 10% energy sav...,"[{'role': 'user', 'content': 'What is the ener...","[energy, saving, plan, city, energy, saving, p...","[0.0016732686, 0.0020316998, 0.0039005126, -0....",7
132,"[{'role': 'user', 'content': 'Who operates the...",When does the concession end?,When does VINCI Airports' concession for opera...,The operating contract for Atlantic City Inter...,The concession for the Atlantic City Internati...,"[{'role': 'user', 'content': 'Who operates the...","[operates, atlantic, city, international, airp...","[0.0010492854, -0.0014025162, 0.0005625584, -0...",7




Cluster 8:


,history,question,query_modifier_question,generated_answer,answer,text,tokens,doc_vector,cluster
129,"[{'role': 'user', 'content': 'Can you tell me ...",What about the airports in Portugal?,What are some airports in Portugal?,Some airports in Portugal include Aéroport de ...,"In Portugal, there are several airports such a...","[{'role': 'user', 'content': 'Can you tell me ...","[tell, airports, airports, including, aéroport...","[-0.004776806, 0.004311443, 0.0027643198, 0.00...",8
134,"[{'role': 'user', 'content': 'Can you tell me ...",What about in Brazil?,What are some airports in Brazil?,"Brazil has several airports, including Aéropor...","In Brazil, there are airports like Tefé Airpor...","[{'role': 'user', 'content': 'Can you tell me ...","[tell, airports, united, airports, united, inc...","[-0.0027654788, 0.0039025743, -0.0013099692, 0...",8
135,"[{'role': 'user', 'content': 'Can you tell me ...",What about airports in Portugal?,What are some airports in Portugal?,"Portugal has several airports, including Aérop...","In Portugal, there are airports like Aéroport ...","[{'role': 'user', 'content': 'Can you tell me ...","[tell, airports, airports, france, aéroport, p...","[-0.0048060925, 0.0047085225, 0.00311829, 0.00...",8




Cluster 9:


,history,question,query_modifier_question,generated_answer,answer,text,tokens,doc_vector,cluster
97,"[{'role': 'user', 'content': 'What can I bring...",What if I forget to leave something prohibited...,What happens if prohibited items are found in ...,"If prohibited items are found in luggage, Lyon...",If you forget to leave a prohibited item at ho...,"[{'role': 'user', 'content': 'What can I bring...","[bring, check, dgac, website, information, pro...","[-0.0010920127, 0.0023740272, -0.0024049792, -...",9
96,"[{'role': 'user', 'content': 'What should I be...",Where can I find the estimated wait time?,Where can I find the estimated wait time for a...,You can find the estimated wait time for airpo...,You can find the estimated wait time for secur...,"[{'role': 'user', 'content': 'What should I be...","[aware, regarding, security, checks, aware, de...","[0.0016338062, 0.002085438, 0.0025325192, 0.00...",9
101,"[{'role': 'user', 'content': 'Can I recharge m...",What should I bring to use the charging stations?,What is required to use the electric vehicle c...,To use the electric vehicle charging stations ...,You need to bring your own adapter to use the ...,"[{'role': 'user', 'content': 'Can I recharge m...","[recharge, electric, vehicle, lyon, recharge, ...","[0.0010020903, 0.005290419, -0.00027900992, 0....",9
99,"[{'role': 'user', 'content': 'Is the parking o...",Where is it located?,Where is the parking located?,The parking is located in an exterior zone and...,The parking is located at Parking P3 at Lyon A...,"[{'role': 'user', 'content': 'Is the parking o...","[parking, open, parking, open, 7, days, parkin...","[0.0011880384, 0.0034522908, 0.002829211, 0.00...",9
0,"[{'role': 'user', 'content': 'What are the mai...",What other sectors are they involved in?,What other sectors is Bouygues involved in bes...,"Besides construction, real estate, media, and ...","Besides construction and real estate, Bouygues...","[{'role': 'user', 'content': 'What are the mai...","[main, business, activities, main, business, a...","[-0.0015435533, 0.0016477973, -0.0029323648, -...",9
